In [ ]:
# import os
# import torch
# import numpy as np
# import cv2
# import matplotlib.pyplot as plt
# from matplotlib.patches import Circle

# # --- PATCH torch.load SAFELY FOR LEGACY CHECKPOINTS ---
# import numpy.core.multiarray
# import functools

# PROJECT_ROOT = '/home/abradshaw/posestimation/elif_model'

# if not hasattr(torch.load, "_is_patched"):
#     _original_torch_load = torch.load

#     @functools.wraps(_original_torch_load)
#     def patched_torch_load(*args, **kwargs):
#         if 'weights_only' not in kwargs:
#             kwargs['weights_only'] = False
#         return _original_torch_load(*args, **kwargs)

#     patched_torch_load._is_patched = True
#     torch.load = patched_torch_load

# torch.serialization.add_safe_globals([numpy.core.multiarray._reconstruct])

# # --- IMPORTS FROM YOUR PIPELINE ---
# from elif_model.models.PoseEstimator.elif_pose import ElifPose
# from elif_model.codecs.simcc_label import SimCCLabel
# from elif_model.datasets.data_augmentation.formatting import PackPoseInputs

# # --- CONFIG ---
# input_size = (192, 256)  # (width, height)
# model_ckpt = os.path.join(PROJECT_ROOT, 'training', 'elif_pose_model.pth')
# image_path = '/home/abradshaw/posestimation/HICO_test2015_00000854.jpg'
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# codec_cfg = dict(
#     input_size=input_size,
#     sigma=(4.9, 5.66),
#     simcc_split_ratio=2.0,
#     normalize=False,
#     use_dark=False
# )

# # --- LOAD MODEL ---
# model = ElifPose()
# model.load_state_dict(torch.load(model_ckpt, map_location=device, weights_only=False))
# model.to(device)
# model.eval()

# # --- LOAD IMAGE ---
# img = cv2.imread(image_path)
# if img is None:
#     raise FileNotFoundError(f'Could not load image: {image_path}')
# h, w = img.shape[:2]

# # --- COMPUTE AFFINE TRANSFORM MANUALLY ---
# src_center = np.array([w / 2, h / 2], dtype=np.float32)
# src_size = np.array([w, h], dtype=np.float32)
# dst_size = np.array(input_size, dtype=np.float32)

# src_points = np.array([
#     src_center,
#     src_center + [0, src_size[1] * -0.5],
#     src_center + [src_size[0] * 0.5, 0]
# ], dtype=np.float32)

# dst_center = dst_size / 2
# dst_points = np.array([
#     dst_center,
#     dst_center + [0, dst_size[1] * -0.5],
#     dst_center + [dst_size[0] * 0.5, 0]
# ], dtype=np.float32)

# warp_mat = cv2.getAffineTransform(src_points, dst_points)
# img_resized = cv2.warpAffine(img, warp_mat, input_size, flags=cv2.INTER_LINEAR)

# # --- PACK INPUT ---
# packed = {
#     'img': img_resized,
#     'img_shape': img_resized.shape,
#     'input_size': input_size
# }
# packed = PackPoseInputs(packed)
# input_tensor = packed['inputs'].unsqueeze(0).float().to(device)

# # --- INFERENCE ---
# with torch.no_grad():
#     simcc_x, simcc_y = model(input_tensor)
#     simcc_x = simcc_x.cpu().numpy()
#     simcc_y = simcc_y.cpu().numpy()
#     decoder = SimCCLabel(**codec_cfg)
#     keypoints, scores = decoder.decode(simcc_x, simcc_y)

# # --- BACK-TRANSFORM TO ORIGINAL IMAGE ---
# warp_mat_inv = cv2.invertAffineTransform(warp_mat)
# keypoints_2d = keypoints[0].astype(np.float32).reshape(-1, 1, 2)
# keypoints_orig = cv2.transform(keypoints_2d, warp_mat_inv).reshape(-1, 2)
# print("Decoded keypoints (x, y):")
# print(keypoints)
# print("Scores:")
# print(scores)
# print(f"Total keypoints: {len(keypoints)}")
# print("Final transformed keypoints shape:", keypoints_orig.shape)



# # --- VISUALIZE ---
# plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
# for (x, y) in keypoints_orig:
#     plt.gca().add_patch(Circle((x, y), 3, color='red'))
# plt.title("Predicted Keypoints")
# plt.axis('off')
# plt.show()


In [ ]:
# import os
# import torch
# import numpy as np
# import cv2
# import matplotlib.pyplot as plt
# from matplotlib.patches import Circle
# import functools
# import numpy.core.multiarray

# # --- Inline fix_aspect_ratio ---
# def fix_aspect_ratio(scale, aspect_ratio):
#     """Adjust the scale to match the given aspect ratio."""
#     w, h = scale[0]
#     expected_w = h * aspect_ratio
#     scale[0][0] = expected_w
#     return scale

# # --- PATCH torch.load for legacy checkpoints ---
# if not hasattr(torch.load, "_is_patched"):
#     _original_torch_load = torch.load
#     @functools.wraps(_original_torch_load)
#     def patched_torch_load(*args, **kwargs):
#         if 'weights_only' not in kwargs:
#             kwargs['weights_only'] = False
#         return _original_torch_load(*args, **kwargs)
#     patched_torch_load._is_patched = True
#     torch.load = patched_torch_load

# torch.serialization.add_safe_globals([numpy.core.multiarray._reconstruct])

# # --- IMPORTS ---
# from elif_model.models.PoseEstimator.elif_pose import ElifPose
# from elif_model.codecs.simcc_label import SimCCLabel
# from elif_model.datasets.data_augmentation.data_augmentation import GetBBoxCenterScale, TopDownAffine
# from elif_model.datasets.data_augmentation.formatting import PackPoseInputs
# from elif_model.datasets.transforms import bbox_xyxy2cs, get_warp_matrix


# # --- CONFIG ---
# PROJECT_ROOT = '/home/abradshaw/posestimation/elif_model'
# input_size = (192, 256)
# images_dir = '/home/abradshaw/posestimation/input_images'  # Folder of input images
# model_ckpt = os.path.join(PROJECT_ROOT, 'training', 'elif_pose_model.pth')
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Codec config
# codec_cfg = dict(
#     input_size=input_size,
#     sigma=(4.9, 5.66),
#     simcc_split_ratio=2.0,
#     normalize=False,
#     use_dark=False
# )

# # --- LOAD MODEL ---
# model = ElifPose()
# model.load_state_dict(torch.load(model_ckpt, map_location=device))
# model.to(device)
# model.eval()

# # --- Loop through all images in folder ---
# valid_exts = ['.jpg', '.jpeg', '.png', '.bmp']
# image_files = sorted([f for f in os.listdir(images_dir) if os.path.splitext(f)[1].lower() in valid_exts])

# for idx, filename in enumerate(image_files):
#     print(f"\n=== Processing Image {idx+1}/{len(image_files)}: {filename} ===")
#     image_path = os.path.join(images_dir, filename)
#     img = cv2.imread(image_path)
#     if img is None:
#         print(f" Skipping {filename}: image not found or unreadable.")
#         continue
#     h, w = img.shape[:2]

#     # --- Define bounding box (same manual values for all) ---
#     x1, y1 = 250,  200
#     x2, y2 = 1700, 2200
#     bbox = np.array([[x1, y1, x2, y2]], dtype=np.float32)

#     # --- PREPROCESSING ---
#     sample = {
#         'img': img,
#         'img_shape': img.shape,
#         'bbox': bbox,
#         'bbox_scores': np.array([[1.0]], dtype=np.float32)
#     }
#     sample = GetBBoxCenterScale(sample)
#     sample['bbox_scale'] = fix_aspect_ratio(sample['bbox_scale'], input_size[0] / input_size[1])
#     center = sample['bbox_center'][0]
#     scale = sample['bbox_scale'][0]
#     rot = sample.get('bbox_rotation', np.zeros((1,), dtype=np.float32))[0]
#     warp_mat = get_warp_matrix(center, scale, rot, output_size=input_size)
#     sample['warp_mat'] = warp_mat
#     sample = TopDownAffine(sample, input_size)

#     # --- Prepare model input ---
#     packed = PackPoseInputs(sample)
#     input_tensor = packed['inputs'].unsqueeze(0).float().to(device)
#     data_sample = packed['data_samples']

#     # --- INFERENCE ---
#     with torch.no_grad():
#         simcc_x, simcc_y = model(input_tensor)
#         decoder = SimCCLabel(**codec_cfg)
#         keypoints, scores = decoder.decode(simcc_x.cpu().numpy(), simcc_y.cpu().numpy())

#     # --- Back-project keypoints ---
#      # --- Back-project keypoints ---
#     keypoints_xy = keypoints[0, :, :2].astype(np.float32).reshape(-1, 1, 2)
#     keypoints_transformed = cv2.transform(keypoints_xy, cv2.invertAffineTransform(warp_mat)).reshape(-1, 2)
#     keypoints_px = np.round(keypoints_transformed).astype(int)
#     keypoints_px[:, 0] = np.clip(keypoints_px[:, 0], 0, w - 1)
#     keypoints_px[:, 1] = np.clip(keypoints_px[:, 1], 0, h - 1)

#     # --- Print keypoints ---
#     print("keypoints = np.array([")
#     for x, y in keypoints_px:
#         print(f"    [{x}, {y}],")
#     print("])")

#     # --- Summary Statistics ---
#     print("\nSummary statistics for keypoints:")
#     print(f"  min x: {keypoints_px[:, 0].min()}  max x: {keypoints_px[:, 0].max()}")
#     print(f"  min y: {keypoints_px[:, 1].min()}  max y: {keypoints_px[:, 1].max()}")
#     print(f"  image dims: width={w}, height={h}")
#     print(f"  total valid keypoints: {len(keypoints_px)}")
#     print("Scores:")
#     print(scores)
#     print("Final transformed keypoints shape:", keypoints_transformed.shape)
#     print(f"Total keypoints: {len(keypoints)} | Unique: {np.unique(keypoints_px, axis=0).shape[0]}")

#     # --- VISUALIZE ---
#     plt.figure(figsize=(8, 10))
#     plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
#     for i, (x, y) in enumerate(keypoints_transformed):
#         plt.gca().add_patch(Circle((x, y), 7, color='red', alpha=0.8))
#         plt.text(x + 5, y, str(i), fontsize=9, color='yellow', weight='bold')
#     bbox_width, bbox_height = x2 - x1, y2 - y1
#     plt.gca().add_patch(plt.Rectangle((x1, y1), bbox_width, bbox_height, linewidth=2, edgecolor='blue', facecolor='none'))
#     plt.title(f"Keypoints: {filename}")
#     plt.axis('off')
#     plt.show()




In [ ]:
# workgin scior for keypints dectionss

import os
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Circle

# --- PATCH torch.load SAFELY FOR LEGACY CHECKPOINTS ---
import numpy.core.multiarray
import functools

PROJECT_ROOT = '/home/abradshaw/posestimation/elif_model'

if not hasattr(torch.load, "_is_patched"):
    _original_torch_load = torch.load

    @functools.wraps(_original_torch_load)
    def patched_torch_load(*args, **kwargs):
        if 'weights_only' not in kwargs:
            kwargs['weights_only'] = False
        return _original_torch_load(*args, **kwargs)

    patched_torch_load._is_patched = True
    torch.load = patched_torch_load

torch.serialization.add_safe_globals([numpy.core.multiarray._reconstruct])

# --- IMPORTS FROM YOUR PIPELINE ---
from elif_model.models.PoseEstimator.elif_pose import ElifPose
from elif_model.codecs.simcc_label import SimCCLabel
from elif_model.datasets.data_augmentation.formatting import PackPoseInputs

# --- CONFIG ---
input_size = (192, 256)  # (width, height)
model_ckpt = os.path.join(PROJECT_ROOT, 'training', 'elif_pose_model.pth')
image_path = '/home/abradshaw/posestimation/input_images/frame_000228.png'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
codec_cfg = dict(
    input_size=input_size,
    sigma=(4.9, 5.66),
    simcc_split_ratio=2.0,
    normalize=False,
    use_dark=False
)

# --- HALPE-26 names (26 joints) ---
HALPE26_NAME = {
  0:"neck", 1:"head_top_mid", 2:"head_left", 3:"head_right",
  4:"l_shoulder_tip", 5:"r_shoulder_tip",
  6:"l_upperarm_root", 7:"r_upperarm_root",
  8:"l_wrist", 9:"r_wrist", 10:"l_elbow",
  11:"r_hip", 12:"l_hip",
  13:"r_knee", 14:"l_knee",
  15:"r_ankle", 16:"l_ankle",
  17:"head_top", 18:"r_clavicle", 19:"pelvis",
  20:"r_heel", 21:"l_small_toe", 22:"r_big_toe",
  23:"l_big_toe", 24:"r_small_toe", 25:"l_heel",
}

# --- LOAD MODEL ---
model = ElifPose()
model.load_state_dict(torch.load(model_ckpt, map_location=device, weights_only=False))
model.to(device)
model.eval()

# --- LOAD IMAGE ---
img = cv2.imread(image_path)
if img is None:
    raise FileNotFoundError(f'Could not load image: {image_path}')
H_img, W_img = img.shape[:2]

# --- COMPUTE AFFINE TRANSFORM (center crop/scale to input_size) ---
src_center = np.array([W_img / 2, H_img / 2], dtype=np.float32)
src_size = np.array([W_img, H_img], dtype=np.float32)
dst_size = np.array(input_size, dtype=np.float32)

src_points = np.array([
    src_center,
    src_center + [0, src_size[1] * -0.5],
    src_center + [src_size[0] * 0.5, 0]
], dtype=np.float32)

dst_center = dst_size / 2
dst_points = np.array([
    dst_center,
    dst_center + [0, dst_size[1] * -0.5],
    dst_center + [dst_size[0] * 0.5, 0]
], dtype=np.float32)

warp_mat = cv2.getAffineTransform(src_points, dst_points)         # 2x3
img_resized = cv2.warpAffine(img, warp_mat, input_size, flags=cv2.INTER_LINEAR)

# --- PACK INPUT ---
packed = {
    'img': img_resized,
    'img_shape': img_resized.shape,
    'input_size': input_size
}
packed = PackPoseInputs(packed)
input_tensor = packed['inputs'].unsqueeze(0).float().to(device)

# --- INFERENCE ---
with torch.no_grad():
    simcc_x, simcc_y = model(input_tensor)
    simcc_x = simcc_x.cpu().numpy()
    simcc_y = simcc_y.cpu().numpy()
    decoder = SimCCLabel(**codec_cfg)
    keypoints, scores = decoder.decode(simcc_x, simcc_y)  # <- fixed (removed stray 'a')

# --- BACK-TRANSFORM TO ORIGINAL IMAGE ---
warp_mat_inv = cv2.invertAffineTransform(warp_mat)
keypoints_2d = keypoints[0].astype(np.float32).reshape(-1, 1, 2)
keypoints_orig = cv2.transform(keypoints_2d, warp_mat_inv).reshape(-1, 2)  # (26,2)
scores_vec = scores[0] if scores is not None else np.full((keypoints_orig.shape[0],), np.nan, dtype=np.float32)

# --- PRINT META ---
print("\n=== Pose export (HALPE-26) ===")
print(f"image_path        : {image_path}")
print(f"rgb_size (HxW)    : {H_img} x {W_img}")
print(f"pose_input_size   : {input_size[1]} x {input_size[0]} (HxW)")
print("affine (2x3)      :")
print(np.array_str(warp_mat, precision=4, suppress_small=True))
print("affine_inv (2x3)  :")
print(np.array_str(warp_mat_inv, precision=4, suppress_small=True))

# --- PRINT TABLE (idx, name, x, y, score) ---
print("\nidx  name               x_rgb      y_rgb      score")
print("---- -----------------  ---------  ---------  ------")
for i, ((x,y), s) in enumerate(zip(keypoints_orig, scores_vec)):
    nm = HALPE26_NAME.get(i, str(i))
    print(f"{i:>2d}   {nm:17s}  {x:9.2f}  {y:9.2f}  {float(s):6.3f}")

# --- (optional) save CSV next to the image ---
csv_path = os.path.splitext(image_path)[0] + "_halpe26_xy.csv"
try:
    with open(csv_path, "w") as f:
        f.write("idx,name,x_rgb,y_rgb,score\n")
        for i, ((x,y), s) in enumerate(zip(keypoints_orig, scores_vec)):
            nm = HALPE26_NAME.get(i, str(i))
            f.write(f"{i},{nm},{float(x):.3f},{float(y):.3f},{float(s):.4f}\n")
    print(f"\n[✓] wrote {csv_path}")
except Exception as e:
    print(f"\n[!] could not write CSV: {e}")

# --- VISUALIZE (with labels) ---
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
for i, (x, y) in enumerate(keypoints_orig):
    plt.gca().add_patch(Circle((x, y), 3, color='red'))
    nm = HALPE26_NAME.get(i, str(i))
    plt.text(x+3, y-3, f"{i}:{nm}", color='yellow', fontsize=7, weight='bold')
plt.title("Predicted Keypoints (HALPE-26, original RGB frame)")
plt.axis('off')
plt.show()
